# Assignment 6
Linda Koine, Jens Weimann, Markus Wiktorin

## 1

## 2

## 3

In [127]:
import neurolab as nl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def func1(x):
    return 1/x

def func2(x):
    return np.log10(x)

def func3(x):
    return np.exp(-x)

def func4(x):
    return np.sin(x)

In [172]:
class mapping:
    def __init__(self, function, range_, str_):
        self.function = function
        self.range_ = range_
        self.str = str_
        
    def generate_set(self, size):
        result = []
        for i in range(size):
            x = np.random.random() * (self.range_[1] - self.range_[0]) + self.range_[0]
            result.append(np.array([x, self.function(x)]))
        return np.array((result))

In [114]:
def get_error(net, test_set):
    error = 0
    for test in test_set:
        error = error + abs(test[1] - net.sim(np.array([[test[0]]])))[0][0]
    return error / np.size(test_set, 0)

In [ ]:
training_size = 100
test_size = 20
hidden_neurons = [1, 2, 3, 4, 5, 10, 20, 50]

mappings = []
mappings.append(mapping(func1, [1,100], "1/x"))
mappings.append(mapping(func2, [1,10], "log_10(x)"))
mappings.append(mapping(func3, [1,10], "exp(-x)"))
mappings.append(mapping(func4, [1, np.pi / 2], "sin(x)"))

for m in mappings:
    training_set = m.generate_set(training_size)
    test_set = m.generate_set(test_size)
    input = training_set[:,0].reshape(training_size, 1)
    target = training_set[:,1].reshape(training_size, 1)
    
    errors = np.zeros((len(hidden_neurons), 1))
    
    hn_idx = 0
    for num_hidden_neurons in hidden_neurons:
        net = nl.net.newff([m.range_], [num_hidden_neurons,1])
        net.train(input, target, epochs=training_size, show=0)
        errors[hn_idx] = get_error(net, test_set)
        hn_idx = hn_idx + 1
    
    _ = plt.plot(errors)
    
_ = plt.legend([m.str for m in mappings], loc=1)
_ = plt.xticks(range(len(hidden_neurons)), hidden_neurons)
_ = plt.xlabel("Number of hidden neurons")
_ = plt.ylabel("Error")